In [1]:
 
import torch
model = torch.hub.load('mateuszbuda/brain-segmentation-pytorch', 'unet',
    in_channels=3, out_channels=1, init_features=32, pretrained=True)

/home/avashchilko/abbyy9sem/course_cvdl/.venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using cache found in /home/avashchilko/.cache/torch/hub/mateuszbuda_brain-segmentation-pytorch_master


In [2]:
from task1pack.utils.data import SegmentationDataset

#pip install -e .
#pip install  abbyy_course_cvdl_t2

from pathlib import Path
from course_ocr_t1.data import MidvPackage
from tqdm import tqdm
from matplotlib import pyplot as plt
import numpy as np
import wandb

import torch

from task1pack.utils.data import HeatmapDataset

In [3]:
#DATASET_PATH = Path() / '..' / '..' / 'midv500_compressed'
DATASET_PATH = Path() / '..' / '..' / 'data' / 'midv500_compressed'
#DATASET_PATH = Path() / '..' / '..' / '..' / '..' / '..' / '..' / 'Downloads' / 'midv500_compressed'
assert DATASET_PATH.exists(), DATASET_PATH.absolute()

# Собираем список пакетов (MidvPackage) 
data_packs = MidvPackage.read_midv500_dataset(DATASET_PATH)
len(data_packs), type(data_packs[0])

(50, course_ocr_t1.data.MidvPackage)

In [4]:
from torchvision.transforms import Resize, Compose, ToTensor

IMAGE_SIZE = [512, 512]

image_transforms = Compose([
    ToTensor(),
    Resize(IMAGE_SIZE),
])

target_transforms = Compose([
    Resize(IMAGE_SIZE),
])

train_dataset = SegmentationDataset(data_packs=data_packs, 
        split='train', image_transforms=image_transforms, target_transforms=target_transforms)
test_dataset = SegmentationDataset(data_packs=data_packs, 
        split='test', image_transforms=image_transforms, target_transforms=target_transforms)

print(len(train_dataset), len(test_dataset))
print(train_dataset[0][0].shape, train_dataset[0][1].shape)

10750 4250
torch.Size([3, 512, 512]) torch.Size([1, 512, 512])


In [5]:
from task1pack.utils.train import train_model, show_train_plots, train_new

In [6]:
from torch.nn import BCELoss

In [8]:
model_name = 'torch unet'

train_dataloader_config = {
    'batch_size': 8,
    'shuffle': True,
    'num_workers': 2,
}

test_dataloader_config = {
    'batch_size': 8,
    'shuffle': False,
    'num_workers': 2,
}

training_config = {
    'lr': 1e-3,
    'epochs': 100,
    'step_size': 25,
    'milestones': [25, 50, 75],
    'gamma': 0.7,
}


device = 'cuda:1'
criterion = BCELoss()

wandb.init(
    project='ocr task 1',
    name='{} {} epochs with lr={} new train'.format(model_name, training_config['epochs'], training_config['lr']),
    config={
        'train_dataloader_config': train_dataloader_config,
        'test_dataloader_config': test_dataloader_config,
        'training_config': training_config,

    "architecture": model_name,
    "dataset": "MIDV-500",
    "criterion": "BCELoss",
    "optimizer": "Adam",
    "image_size": IMAGE_SIZE,  
    }
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: vashchilkoav. Use `wandb login --relogin` to force relogin


In [9]:
train_losses, test_losses, trained_model = train_new(
    train_dataset=train_dataset, 
    test_dataset=test_dataset, 
    model=model, 
    train_dataloader_kwargs=train_dataloader_config, 
    test_dataloader_kwargs=test_dataloader_config, 
    training_kwargs=training_config,
    criterion=criterion,
    device=device,
    wandb_instance=wandb,
)

Epoch 1:
Epoch 2:


KeyboardInterrupt: 

In [ ]:
torch.save(trained_model.state_dict(), './unet.pth')

In [ ]:
wandb.finish()

In [7]:
model_name = 'torch unet'

train_dataloader_config = {
    'batch_size': 8,
    'shuffle': True,
}

test_dataloader_config = {
    'batch_size': 8,
    'shuffle': False,
}

training_config = {
    'lr': 1e-3,
    'epochs': 100,
    'step_size': 25,
    'gamma': 0.7,
}


device = 'cuda:1'
criterion = BCELoss()

wandb.init(
    project='ocr task 1',
    name='{} {} epochs with lr={} old train'.format(model_name, training_config['epochs'], training_config['lr']),
    config={
        'train_dataloader_config': train_dataloader_config,
        'test_dataloader_config': test_dataloader_config,
        'training_config': training_config,

    "architecture": model_name,
    "dataset": "MIDV-500",
    "criterion": "BCELoss",
    "optimizer": "Adam",
    "image_size": IMAGE_SIZE,  
    }
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: vashchilkoav. Use `wandb login --relogin` to force relogin


In [8]:
from task1pack.utils.train import train_old

In [9]:
trained_model = train_old(
    train_dataset, 
    net=model, 
    criterion=criterion, 
    batch_size=train_dataloader_config['batch_size'], 
    epochs=training_config['epochs'],
    lr=training_config['lr'], 
    device=device,
    wandb_instance=wandb
)

epoch 0|0; total loss:0.024599019438028336
last losse: 3.3208677768707275
epoch 0|135; total loss:0.26201707124710083
last losse: 0.203396275639534
epoch 0|270; total loss:0.10895795375108719
last losse: 0.06776674091815948
epoch 0|405; total loss:0.07589032500982285
last losse: 0.12207520008087158
epoch 0|540; total loss:0.06147361174225807
last losse: 0.08387358486652374
epoch 0|675; total loss:0.04958047345280647
last losse: 0.06973102688789368
epoch 0|810; total loss:0.04651539400219917
last losse: 0.08303708583116531
epoch 0|945; total loss:0.039703369140625
last losse: 0.022554796189069748
epoch 0|1080; total loss:0.0352974496781826
last losse: 0.013174505904316902
epoch 0|1215; total loss:0.0334409661591053
last losse: 0.014591091312468052
epoch 1|0; total loss:0.00013088210835121572
last losse: 0.017669085413217545
epoch 1|135; total loss:0.026703229174017906
last losse: 0.014996595680713654
epoch 1|270; total loss:0.027345990762114525
last losse: 0.018553050234913826
epoch 1|4

KeyboardInterrupt: 

In [10]:
wandb.finish()

lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr,0.001


In [ ]:
torch.save(trained_model.state_dict(), './unet.pth')